# Inferring connectivity using covariance based methods 

In this study, we inferred connectivity using
1. Network deconvolution (ND)
2. Graphical LASSO (Glasso)
3. Differential covariance (Dcov)
4. Dynamic differential covariance (DDC)
5. Fractional differential covariance (FDDC)

Please refer to the manuscript for appropriate references.

(*for pairwise inference using CCG, please check the 'matlab' folder.)

In [1]:
# setting up

import sys
import numpy as np
sys.path.append('./python/')
sys.path.append('../util')

import coninf
import util

# Load spike trains (here we use a simulation of 5 sec. purely for demonstration purpose, please consider using longer recordings/simulations) 
data_path = '../example_data/2000/2000_13.0_24.0_5.0/instances/0.npy'
data = np.load(data_path, allow_pickle=True).item()

# get spiketrains and groundtruth connections for inference
spktrain = util.get_spktrains(data_path)
gt_conn_dict = util.get_gt_conn(data)


In [11]:
# print out spktrain variable
print(spktrain) 
print('number of active neurons in this simulation : ' + str(len(np.unique(spktrain['ids'])))) # 3 neurons were dormant for this short simulation
# ids : spike template ids (neuron id)
# spktrains : spike times in ms

{'ids': array([679,  48, 618, ..., 200, 340, 784], dtype=int32), 'spktrains': array([   0. ,    0. ,    0. , ..., 3999.9, 3999.9, 3999.9])}
number of active neurons in this simulation : 1997


In [13]:
# print out groundtruth connections
print(gt_conn_dict['all'])
print('number of neurons that had at least one connection in this simulation : '+ str(len(np.unique(gt_conn_dict['all']))))

[[   0    0    0 ... 1999 1999 1999]
 [  54   93  122 ... 1515 1529 1595]]
number of neurons that had at least one connection in this simulation : 2000


# Network deconvolution

In [4]:
# setting parameters

params = {}
# method to use
params['method']='nd'
# length of data to use 
params['t_compute'] = int(4) # in seconds 
# set spike train data
params['spktrains']= spktrain['spktrains']
params['ids']=spktrain['ids']
params['gt_conn']=gt_conn_dict

# ND specific params
params['covar_bin'] = int(5) # in miliseconds. 
params['beta'] = 0.99 # for netowrk deconv.


nd_result = coninf.run_inference(params)


/home/takim/.local/lib/python3.7/site-packages/elephant/conversion.py:835: UserWarning: Correcting 1 rounding errors by shifting the affected spikes into the following bin. You can set tolerance=None to disable this behaviour.
  'behaviour.'.format(num_rounding_corrections))


decomposition and deconvolution...
inference successfully finished


In [5]:
# quick check of the result (*same for all follwing methods)
nd_result.keys()
# true_con : adjacency matrix (groundtruth connections)
# thres_con : not used (for now same as true_con)
# score_con : adjacency matrix (estimated score)
# compute_time : cpu wall clock time (in seconds)
# sol_mat : raw output of the method (i.e. before taking absolute values)
# method_params : returns params that were used for the inference.

dict_keys(['true_con', 'thres_con', 'score_con', 'compute_time', 'sol_mat', 'method_params'])

In [6]:
# evaluating inference performance (*same for all following methods)
eval_result = util.eval_performance(nd_result)
eval_result 

,compute_time,fpr,tpr,thresholds,prc_precision,prc_recall,prc_thresholds,auc,aps
0,8.576842,"[0.0, 2.560198589484189e-07, 1.536119153690513...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.2489851995253857e-...","[2.0, 1.0, 0.9753079445722413, 0.9684357774854...","[0.02008649246414712, 0.019785660742561745, 0....","[1.0, 0.6863673265471805, 0.6863673265471805, ...","[0.0, 0.7639089868516508, 0.7639089868516517, ...",0.519834,0.023083


# Graphical LASSO

In [7]:
# setting parameters

params = {}
# method to use
params['method']='glasso'
# length of data to use 
params['t_compute'] = int(4) # in seconds 
# set spike train data
params['spktrains']= spktrain['spktrains']
params['ids']=spktrain['ids']
params['gt_conn']=gt_conn_dict

# glasso specific params
params['covar_bin'] = int(5) # for binning
params['gamma']=0.1 # regularization parameter


glasso_result = coninf.run_inference(params)

/home/takim/.local/lib/python3.7/site-packages/elephant/conversion.py:835: UserWarning: Correcting 1 rounding errors by shifting the affected spikes into the following bin. You can set tolerance=None to disable this behaviour.
  'behaviour.'.format(num_rounding_corrections))


Shape of empirical covariance matrix:  (1997, 1997)
Length of the sampled time series:  800
 
SINGLE GRAPHICAL LASSO PROBLEM 
Regularization parameters:
{'lambda1': 0.05, 'mu1': None}
ADMM terminated after 42 iterations with status: optimal.
ADMM terminated after 11 iterations with status: optimal.
ADMM terminated after 68 iterations with status: optimal.
ADMM terminated after 85 iterations with status: optimal.
{'lambda1': 0.03162277660168379, 'mu1': 0}
inference successfully finished


# Differential covariance

In [8]:
# setting parameters

params = {}
# method to use
params['method']='dcov'
# length of data to use 
params['t_compute'] = int(4) # in seconds 
# set spike train data
params['spktrains']= spktrain['spktrains']
params['ids']=spktrain['ids']
params['gt_conn']=gt_conn_dict

# Dcov specific params
params['covar_bin'] = int(5) # for binning


dcov_result = coninf.run_inference(params)

/home/takim/.local/lib/python3.7/site-packages/elephant/conversion.py:835: UserWarning: Correcting 1 rounding errors by shifting the affected spikes into the following bin. You can set tolerance=None to disable this behaviour.
  'behaviour.'.format(num_rounding_corrections))


inference successfully finished


# Dynamic differential covariance

In [9]:
# setting parameters

params = {}
# method to use
params['method']='ddc'
# length of data to use 
params['t_compute'] = int(4) # in seconds 
# set spike train data
params['spktrains']= spktrain['spktrains']
params['ids']=spktrain['ids']
params['gt_conn']=gt_conn_dict

# Dcov specific params
params['covar_bin'] = int(5) # for binning
params['kernel']= 'gauss' # 'alpha' or 'gauss'
params['kernel_size']=int(3) # in ms. 


ddc_result = coninf.run_inference(params)


(1997, 800)
inference successfully finished


# Fractional dynamic differential covariance

In [10]:
# setting parameters

params = {}
# method to use
params['method']='frac_ddc'
# length of data to use 
params['t_compute'] = int(4) # in seconds 
# set spike train data
params['spktrains']= spktrain['spktrains']
params['ids']=spktrain['ids']
params['gt_conn']=gt_conn_dict

# FDDC specific params
params['covar_bin'] = int(5) # for binning
params['kernel']= 'alpha' # 'alpha' or 'gauss'
params['kernel_size']=int(3) # in ms. 
params['frac_order']=float(0.1) # fractional order ('beta' in the manuscript)
params['diff_window']=int(10) # how many terms will be used to compute fractional differentiation, k=10 in the manuscript.

# if frac_order is not set (perform Adfuller test on sampled neurons): 
params['sample_n']= int(100) # number of neurons to sample 
params['sample_window']= int(300) # time window (number of bins) to sample 


fddc_result = coninf.run_inference(params)




(1997, 800)
computing fractional diff with dimension of 0.1
inference successfully finished
